pca_bert

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings 
warnings.filterwarnings("ignore") 

In [3]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Bert/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Bert/bert_withstopwordchangelevels_50_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,0,-15.959655,7.610355,-4.614255,3.683309,2.789502,-1.250055,-1.805187,-10.514873,-4.086176,...,-1.186375,0.159350,-1.093152,0.330839,-0.894613,0.428084,0.338784,-0.150229,1.068460,0.249122
1,1,-13.495674,4.785825,-0.513378,4.535705,-0.666486,-2.018108,-2.203631,5.270856,-0.704172,...,0.361583,0.478075,-0.118859,-0.141411,-0.203695,0.034060,0.600824,0.582139,0.334277,-0.339090
2,2,5.556126,-6.071359,15.245860,-6.528944,1.345797,5.089355,-4.129077,-5.236971,-3.814316,...,-1.792032,1.442494,-0.360016,-0.082643,1.296235,-1.427958,-0.955182,-0.505013,0.140971,-1.258967
3,3,-4.445179,1.293825,-4.332579,-6.459947,3.055774,8.987284,7.318152,-5.308832,-3.731958,...,-1.703480,0.010108,-0.972003,0.613516,1.938680,-0.539077,-2.815410,-0.767579,0.535993,-0.797360
4,4,-12.149864,1.734085,-7.929423,-3.487395,3.067365,4.222421,-4.707823,2.094515,0.077916,...,0.962144,-0.990792,0.010149,1.077357,0.384861,-1.077598,-0.134066,-0.113422,-1.149983,-1.057587


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-15.959655,7.610355,-4.614255,3.683309,2.789502,-1.250055,-1.805187,-10.514873,-4.086176,0.203140,...,-1.186375,0.159350,-1.093152,0.330839,-0.894613,0.428084,0.338784,-0.150229,1.068460,0.249122
1,-13.495674,4.785825,-0.513378,4.535705,-0.666486,-2.018108,-2.203631,5.270856,-0.704172,1.341339,...,0.361583,0.478075,-0.118859,-0.141411,-0.203695,0.034060,0.600824,0.582139,0.334277,-0.339090
2,5.556126,-6.071359,15.245860,-6.528944,1.345797,5.089355,-4.129077,-5.236971,-3.814316,-2.739572,...,-1.792032,1.442494,-0.360016,-0.082643,1.296235,-1.427958,-0.955182,-0.505013,0.140971,-1.258967
3,-4.445179,1.293825,-4.332579,-6.459947,3.055774,8.987284,7.318152,-5.308832,-3.731958,-2.567814,...,-1.703480,0.010108,-0.972003,0.613516,1.938680,-0.539077,-2.815410,-0.767579,0.535993,-0.797360
4,-12.149864,1.734085,-7.929423,-3.487395,3.067365,4.222421,-4.707823,2.094515,0.077916,1.247958,...,0.962144,-0.990792,0.010149,1.077357,0.384861,-1.077598,-0.134066,-0.113422,-1.149983,-1.057587


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 50), (524, 50))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3924425092580082


**Decision Tree**

In [16]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [17]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [18]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [19]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [20]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [21]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.486066,0.486066,0.531079,0.485532,0.349237,0.349237,0.40137,0.353116


**Random Forest**

In [22]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [23]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [24]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [25]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3049    0.1592    0.2092       157
           2     0.4463    0.7722    0.5657       237
           3     0.2188    0.0538    0.0864       130

    accuracy                         0.4103       524
   macro avg     0.3233    0.3284    0.2871       524
weighted avg     0.3475    0.4103    0.3400       524



In [26]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.486066,0.486066,0.531079,0.485532,0.349237,0.349237,0.401370,0.353116
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.410305,0.410305,0.347493,0.339977


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5741    0.4294    0.4913       361
           2     0.5445    0.7631    0.6355       553
           3     0.5429    0.3105    0.3950       306

    accuracy                         0.5508      1220
   macro avg     0.5538    0.5010    0.5073      1220
weighted avg     0.5528    0.5508    0.5325      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2975    0.2293    0.2590       157
           2     0.4766    0.6456    0.5484       237
           3     0.2683    0.1692    0.2075       130

    accuracy                         0.4027       524
   macro avg     0.3475    0.3480    0.3383       524
weighted avg     0.3713    0.4027    0.3771       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.512295,0.512295,0.611359,0.439902,0.446565,0.446565,0.370496,0.355045
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.417939,0.417939,0.355411,0.355542
2,Adaboost,0.550820,0.550820,0.552846,0.532526,0.402672,0.402672,0.371281,0.377120


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9691    0.8698    0.9168       361
           2     0.8598    0.9982    0.9238       553
           3     0.9961    0.8268    0.9036       306

    accuracy                         0.9172      1220
   macro avg     0.9417    0.8983    0.9147      1220
weighted avg     0.9263    0.9172    0.9167      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3204    0.2102    0.2538       157
           2     0.4519    0.6540    0.5345       237
           3     0.2436    0.1462    0.1827       130

    accuracy                         0.3950       524
   macro avg     0.3386    0.3368    0.3237       524
weighted avg     0.3608    0.3950    0.3631       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.512295,0.512295,0.611359,0.439902,0.446565,0.446565,0.370496,0.355045
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.417939,0.417939,0.355411,0.355542
2,Adaboost,0.550820,0.550820,0.552846,0.532526,0.402672,0.402672,0.371281,0.377120
3,GBM,0.917213,0.917213,0.926336,0.916674,0.395038,0.395038,0.360814,0.363123


**SVM Model**

In [27]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [28]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [29]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [30]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4089    0.5346    0.4634       361
           2     0.5925    0.3996    0.4773       553
           3     0.3413    0.4183    0.3759       306

    accuracy                         0.4443      1220
   macro avg     0.4476    0.4509    0.4389      1220
weighted avg     0.4752    0.4443    0.4478      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2944    0.4013    0.3396       157
           2     0.4601    0.3165    0.3750       237
           3     0.2245    0.2538    0.2383       130

    accuracy                         0.3263       524
   macro avg     0.3263    0.3239    0.3176       524
weighted avg     0.3520    0.3263    0.3305       524



In [31]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.486066,0.486066,0.531079,0.485532,0.349237,0.349237,0.401370,0.353116
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.410305,0.410305,0.347493,0.339977
2,SVM,0.444262,0.444262,0.475171,0.447764,0.326336,0.326336,0.352008,0.330478


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5093    0.5319    0.5203       361
           2     0.5777    0.7324    0.6459       553
           3     0.4859    0.2255    0.3080       306

    accuracy                         0.5459      1220
   macro avg     0.5243    0.4966    0.4914      1220
weighted avg     0.5345    0.5459    0.5240      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2683    0.2803    0.2741       157
           2     0.4530    0.5696    0.5047       237
           3     0.2419    0.1154    0.1562       130

    accuracy                         0.3702       524
   macro avg     0.3211    0.3218    0.3117       524
weighted avg     0.3453    0.3702    0.3492       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.512295,0.512295,0.611359,0.439902,0.446565,0.446565,0.370496,0.355045
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.417939,0.417939,0.355411,0.355542
2,Adaboost,0.550820,0.550820,0.552846,0.532526,0.402672,0.402672,0.371281,0.377120
3,GBM,0.917213,0.917213,0.926336,0.916674,0.395038,0.395038,0.360814,0.363123
4,SVM,0.458197,0.458197,0.339296,0.297115,0.444656,0.444656,0.272717,0.288721
5,KNN,0.545902,0.545902,0.534455,0.524014,0.370229,0.370229,0.345304,0.349161


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4456    0.3629    0.4000       361
           2     0.4974    0.6962    0.5803       553
           3     0.3092    0.1536    0.2052       306

    accuracy                         0.4615      1220
   macro avg     0.4174    0.4042    0.3952      1220
weighted avg     0.4349    0.4615    0.4329      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2903    0.2293    0.2562       157
           2     0.4660    0.6371    0.5383       237
           3     0.2368    0.1385    0.1748       130

    accuracy                         0.3912       524
   macro avg     0.3311    0.3350    0.3231       524
weighted avg     0.3565    0.3912    0.3636       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.512295,0.512295,0.611359,0.439902,0.446565,0.446565,0.370496,0.355045
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.417939,0.417939,0.355411,0.355542
2,Adaboost,0.550820,0.550820,0.552846,0.532526,0.402672,0.402672,0.371281,0.377120
3,GBM,0.917213,0.917213,0.926336,0.916674,0.395038,0.395038,0.360814,0.363123
4,SVM,0.458197,0.458197,0.339296,0.297115,0.444656,0.444656,0.272717,0.288721
5,KNN,0.545902,0.545902,0.534455,0.524014,0.370229,0.370229,0.345304,0.349161
6,GNB,0.461475,0.461475,0.434871,0.432857,0.391221,0.391221,0.356534,0.363605
